In [1]:
import pandas as pd
import numpy as np
import cv2

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

import matplotlib.pyplot as plt
import os


In [2]:
def dice_coef_9cat(y_true, y_pred, smooth=1e-7):
    '''
    Dice coefficient for 3 categories. Ignores background pixel label 0
    Pass to model as metric during compile statement
    '''
    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=3)[...,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return K.mean((2. * intersect / (denom + smooth)))

def dice_coef_9cat_loss(y_true, y_pred):
    '''
    Dice loss to minimize. Pass to model as loss during compile statement
    '''
    return 1 - dice_coef_9cat(y_true, y_pred)

In [3]:
loss = dice_coef_9cat_loss
segmentation_model = keras.models.load_model('C:\Luna_CS\Aravind\working_model.h5', 
                                   custom_objects={ loss.__name__: loss })

In [4]:
def remove_nerves(image):
    img = array_to_img(image)
    
    img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
    # convert image to grayScale
    grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
   
    # kernel for morphologyEx
    kernel = cv2.getStructuringElement(1,(17,17))
   
    # apply MORPH_BLACKHAT to grayScale image
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
  
    # apply thresholding to blackhat
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)

    # inpaint with original image and threshold image
    final_image = cv2.inpaint(img,threshold,1,cv2.INPAINT_TELEA)
    final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB)
    
    return final_image.astype(np.float64)/255.0

In [5]:
def get_cropped_image(file, model):
    test_img = load_img(file, target_size=(512, 512))
    test_img_original = load_img(file)
    full_arr = np.array(test_img_original)/255.0
    x_true = full_arr.shape[0]
    y_true = full_arr.shape[1]
    test_arr = img_to_array(test_img)/255.0
    
    test_arr_preprocessed = remove_nerves(test_arr)

    pred = model.predict(test_arr_preprocessed.reshape(1, 512, 512, 3))
    single_pred = pred[0]
    mask_img = array_to_img(single_pred)
    mask_img = mask_img.resize((y_true, x_true))
    mask_arr = np.array(mask_img)/255.0
    my_mask = np.zeros((x_true, y_true))
    marked = []
    inner = []
    for i in range(x_true):
        for j in range(y_true):
            if np.argmax(mask_arr[i][j]) != 0:
                marked.append([i, j])
                
    avgi = []
    avgj = []
    for lis in marked:
        first = lis[0]
        sec = lis[1]
        avgi.append(first)
        avgj.append(sec)
        

        
    avgi = sum(avgi)//len(avgi)
    avgj = sum(avgj)//len(avgj)
    
    middle  = (avgi, avgj)
    K = int((300.0/6291456.0)*float(x_true*y_true))
    top = middle[1] - K
    bottom = middle[1] + K
    left = middle[0] - K
    right = middle[0] + K
    
    cropped = test_img_original.crop(((top, left, bottom, right)))

    return np.array(cropped)/255.0
    

In [6]:
df = pd.read_csv('C:\Luna_CS\Aravind\ORIGA\OrigaList.csv')
df.head()

,Eye,Filename,ExpCDR,Set,Glaucoma
0,OD,001.jpg,0.7097,A,0
1,OS,002.jpg,0.6953,A,0
2,OS,003.jpg,0.9629,A,0
3,OD,004.jpg,0.7246,A,0
4,OS,005.jpg,0.6138,A,0


In [7]:
simple_df = df.drop(['Eye', 'ExpCDR', 'Set'], axis=1, inplace=False)

## Add Full Filename Paths ##

folder = 'C:\Luna_CS\Aravind\ORIGA\Images'
simple_df['Filename_full'] = [os.path.join(folder, file) for file in simple_df['Filename']]
simple_df['Cropped'] = [False for file in simple_df['Filename']]
simple_df.head()


,Filename,Glaucoma,Filename_full,Cropped
0,001.jpg,0,C:\Luna_CS\Aravind\ORIGA\Images\001.jpg,False
1,002.jpg,0,C:\Luna_CS\Aravind\ORIGA\Images\002.jpg,False
2,003.jpg,0,C:\Luna_CS\Aravind\ORIGA\Images\003.jpg,False
3,004.jpg,0,C:\Luna_CS\Aravind\ORIGA\Images\004.jpg,False
4,005.jpg,0,C:\Luna_CS\Aravind\ORIGA\Images\005.jpg,False


In [8]:
folder_path = 'C:\Luna_CS\Aravind\Database\Images'
for file in os.listdir(folder_path):
    columns = ['Filename', 'Glaucoma', 'Filename_full', 'Cropped']
    row = pd.DataFrame([[file, int('_g_' in file),os.path.join(folder_path, file), True]],
                      columns = columns)
    simple_df = simple_df.append(row)

In [9]:
simple_df = simple_df.reset_index(drop=True)
simple_df.tail()

,Filename,Glaucoma,Filename_full,Cropped
1350,Im701_g_ACRIMA.jpg,1,C:\Luna_CS\Aravind\Database\Images\Im701_g_ACR...,True
1351,Im702_g_ACRIMA.JPG,1,C:\Luna_CS\Aravind\Database\Images\Im702_g_ACR...,True
1352,Im703_g_ACRIMA.JPG,1,C:\Luna_CS\Aravind\Database\Images\Im703_g_ACR...,True
1353,Im704_g_ACRIMA.jpg,1,C:\Luna_CS\Aravind\Database\Images\Im704_g_ACR...,True
1354,Im705_g_ACRIMA.jpg,1,C:\Luna_CS\Aravind\Database\Images\Im705_g_ACR...,True


In [10]:
sum(simple_df['Glaucoma'])/len(simple_df['Glaucoma'])

0.41623616236162364

In [11]:
g1020_folder = 'C:\Luna_CS\Aravind\G1020\G1020'
g1020_df = pd.read_csv('C:\Luna_CS\Aravind\G1020\G1020\G1020.csv')
clean_df = pd.DataFrame({})
clean_df['Filename'] = g1020_df['imageID']
clean_df['Glaucoma'] = g1020_df['binaryLabels']

clean_df['Filename_full'] = [os.path.join(g1020_folder, img_name) for img_name in g1020_df['imageID']]

clean_df['Cropped'] = [False for _ in g1020_df['imageID']]

In [12]:
simple_df = simple_df.append(clean_df)

In [13]:
simple_df = simple_df.reset_index(drop=True)
simple_df.tail()

,Filename,Glaucoma,Filename_full,Cropped
2370,image_3198.jpg,0,C:\Luna_CS\Aravind\G1020\G1020\image_3198.jpg,False
2371,image_3199.jpg,0,C:\Luna_CS\Aravind\G1020\G1020\image_3199.jpg,False
2372,image_3201.jpg,1,C:\Luna_CS\Aravind\G1020\G1020\image_3201.jpg,False
2373,image_3202.jpg,1,C:\Luna_CS\Aravind\G1020\G1020\image_3202.jpg,False
2374,image_2568.jpg,0,C:\Luna_CS\Aravind\G1020\G1020\image_2568.jpg,False


In [14]:
sum(simple_df['Glaucoma'])/len(simple_df['Glaucoma'])

0.36210526315789476

```simple_df``` now stores the name of each file, which class they belong to, and whether they need to be cropped or not. Based on this information, the next code cell goes through each row of this dataframe, and crops it if necessasry. Then they are placed in the **Healthy** folder if the label is 0, and placed in the **Glaucoma** folder if the label is 1. 

In [ ]:

for file, label, path, cropped in zip(simple_df['Filename'], simple_df['Glaucoma'], 
                                   simple_df['Filename_full'], simple_df['Cropped']):
    if label == 0:
        folder = 'C:\Luna_CS\Aravind\Cropped Images\Healthy'
        if cropped:
            img = load_img(path)
        else:
            img = get_cropped_image(path, segmentation_model)
            img = array_to_img(img)
            
        img = img.save(os.path.join(folder, file))
        
    else:
        folder = 'C:\Luna_CS\Aravind\Cropped Images\Glaucoma'
        if cropped:
            img = load_img(path)
        else:
            img = get_cropped_image(path, segmentation_model)
            img = array_to_img(img)
            
        img = img.save(os.path.join(folder, file))        
